In [1]:
# # %%
# %pip install -U -q transformers bitsandbytes
# %pip install -U -q huggingface_hub
# %pip install -q flask flask-cors pyngrok flash_attn
# %pip install -q transformers bitsandbytes huggingface_hub flask flask-cors pyngrok

# # %%

In [4]:
import numpy as np
import torch
import torchvision.transforms as T
# from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer


c:\Users\hn745\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Ensure protobuf is downgraded before any imports that use it
# %pip install protobuf==3.20.3

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

model = AutoModel.from_pretrained(
    "5CD-AI/Vintern-1B-v3_5",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    use_flash_attn=False,
).eval().cpu()

tokenizer = AutoTokenizer.from_pretrained("5CD-AI/Vintern-1B-v3_5", trust_remote_code=True, use_fast=False)


c:\Users\hn745\AppData\Local\Programs\Python\Python39\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


In [6]:
test_image = "output1.jpg"
pixel_values = load_image(test_image, max_num=6).to(torch.bfloat16).cpu()
generation_config = dict(max_new_tokens= 1024, do_sample=False, num_beams = 3, repetition_penalty=2.5)

question = '''<image>\nTrích xuất thông tin chính trong ảnh theo Họ và tên,'Số thẻ, Ngày sinh, Giới tính, Quốc tịch, Quê quán, Nơi thường trú, Có giá trị đến và trả về định dạng json như sau:
json
{
  "Họ và tên": "...",
  "Số thẻ": "",
  "Ngày sinh": "...",
  "Giới tính": "...",
  "Quốc tịch": "...",
  "Quê quán": "....",
  "Nơi thường trú": "...",
  "Có giá trị đến": "..."
}
 '''

response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1976 > 1700). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


```json
{
  "Họ và tên": "NGUYỄN THANH HOÀN",
  "Số thẻ": "01920409589",
  "Ngày sinh": "25/08/2004",
  "Giới tính": "Nam",
  "Quốc tịch": "Việt Nam",
  "Quê quán": "Thái Nguyên",
  "Nơi thường trú": "Thái Bình, Thái Nguyên",
  "Có giá trị đến": "25/08/2029"
}
```


In [6]:
#pip install unidecode

## Lấy khuôn mặt

In [ ]:
import os
import cv2
import json
import time
from unidecode import unidecode
import csv

# ==== Làm sạch JSON nếu cần ====
response_cleaned = response.strip().strip('`').strip()
if response_cleaned.startswith("json"):
    response_cleaned = response_cleaned[4:].strip()
    
info = json.loads(response_cleaned)

# ==== Tạo folder đích tên: <CCCD>_<TEN_KHONG_DAU> ====
person_name_raw = info["Họ và tên"]
person_name = unidecode(person_name_raw.strip().replace(" ", "_"))
folder_name = f"{info['Số thẻ']}_{person_name}"
full_path = os.path.join(folder_name) 

# ==== Tạo folder chính nếu chưa có ====
os.makedirs(full_path, exist_ok=True)

# ==== Tạo folder con trong full_path với tên giống folder chính ====
subfolder_path = os.path.join(full_path, folder_name)
os.makedirs(subfolder_path, exist_ok=True)

# ==== Ghi thông tin vào CSV trong folder con ====
info_file = os.path.join(subfolder_path, f"info_{info['Số thẻ']}_{person_name_raw}.csv")
with open(info_file, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Họ và tên", "Số thẻ", "Ngày sinh", "Giới tính", "Quốc tịch", "Quê quán","Nơi thường trú", "Có giá trị đến"])
    writer.writerow([
        info['Họ và tên'],
        info['Số thẻ'],
        info['Ngày sinh'],
        info['Giới tính'],
        info['Quốc tịch'],
        info['Quê quán'],  
        info['Nơi thường trú'],  
        info['Có giá trị đến']
    ])

# ==== Mở webcam để chụp ảnh và lưu trong folder con ====
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Không mở được webcam.")
    exit()

count = 0
max_images = 200

while count < max_images:
    ret, frame = cap.read()
    if not ret:
        print("Không thể lấy ảnh từ webcam.")
        break

    frame = cv2.flip(frame, 1)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    img_name = f"{count}.jpg"
    save_path = os.path.join(subfolder_path, img_name)
    if cv2.imwrite(save_path, gray_frame):
        print(f"Đã lưu {img_name}")
    else:
        print(f"Lỗi lưu {img_name}")

    cv2.imshow("Face dataset", frame)
    count += 1
    time.sleep(0.05)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Đã lưu 0.jpg
Đã lưu 1.jpg
Đã lưu 2.jpg
Đã lưu 3.jpg
Đã lưu 4.jpg
Đã lưu 5.jpg
Đã lưu 6.jpg
Đã lưu 7.jpg
Đã lưu 8.jpg
Đã lưu 9.jpg
Đã lưu 10.jpg
Đã lưu 11.jpg
Đã lưu 12.jpg
Đã lưu 13.jpg
Đã lưu 14.jpg
Đã lưu 15.jpg
Đã lưu 16.jpg
Đã lưu 17.jpg
Đã lưu 18.jpg
Đã lưu 19.jpg
Đã lưu 20.jpg
Đã lưu 21.jpg
Đã lưu 22.jpg
Đã lưu 23.jpg
Đã lưu 24.jpg
Đã lưu 25.jpg
Đã lưu 26.jpg
Đã lưu 27.jpg
Đã lưu 28.jpg
Đã lưu 29.jpg
Đã lưu 30.jpg
Đã lưu 31.jpg
Đã lưu 32.jpg
Đã lưu 33.jpg
Đã lưu 34.jpg
Đã lưu 35.jpg
Đã lưu 36.jpg
Đã lưu 37.jpg
Đã lưu 38.jpg
Đã lưu 39.jpg
Đã lưu 40.jpg
Đã lưu 41.jpg
Đã lưu 42.jpg
Đã lưu 43.jpg
Đã lưu 44.jpg
Đã lưu 45.jpg
Đã lưu 46.jpg
Đã lưu 47.jpg
Đã lưu 48.jpg
Đã lưu 49.jpg
Đã lưu 50.jpg
Đã lưu 51.jpg
Đã lưu 52.jpg
Đã lưu 53.jpg
Đã lưu 54.jpg
Đã lưu 55.jpg
Đã lưu 56.jpg
Đã lưu 57.jpg
Đã lưu 58.jpg
Đã lưu 59.jpg
Đã lưu 60.jpg
Đã lưu 61.jpg
Đã lưu 62.jpg
Đã lưu 63.jpg
Đã lưu 64.jpg
Đã lưu 65.jpg
Đã lưu 66.jpg
Đã lưu 67.jpg
Đã lưu 68.jpg
Đã lưu 69.jpg
Đã lưu 70.jpg
Đã lưu 71.jpg
Đã

In [ ]:
# pip install face_recognition



## Encode khuôn mặt

In [ ]:
import face_recognition
import os
import cv2
import pickle

encodings = []
names = []
pkl_file = "face_encodings.pkl"

# ==== Load dữ liệu đã có nếu file .pkl tồn tại ====
if os.path.exists(pkl_file):
    with open(pkl_file, "rb") as f:
        existing_data = pickle.load(f)
        encodings = existing_data.get("encodings", [])
        names = existing_data.get("names", [])
        print(f"Đã nạp {len(encodings)} khuôn mặt đã lưu.")

# ==== Lặp qua các thư mục trong dataset mới ====
for folder_name in os.listdir(full_path):
    person_dir = os.path.join(full_path, folder_name)
    if not os.path.isdir(person_dir):
        continue

    print(f"Đang xử lý: {folder_name}")

    for file_name in os.listdir(person_dir):
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(person_dir, file_name)

        # Đọc ảnh và nhận diện khuôn mặt
        image = face_recognition.load_image_file(img_path)
        face_locations = face_recognition.face_locations(image)

        if len(face_locations) == 0:
            print(f"Không tìm thấy khuôn mặt trong {file_name}")
            continue

        # Nếu đã có encoding của người này và ảnh này thì bỏ qua (tuỳ chọn)
        face_encoding = face_recognition.face_encodings(image, face_locations)[0]
        encodings.append(face_encoding)
        names.append(folder_name)
        print(f"Đã lưu encoding cho {file_name} trong {folder_name}")

# ==== Ghi lại toàn bộ encodings + names vào file .pkl ====
data = {"encodings": encodings, "names": names}
with open(pkl_file, "wb") as f:
    pickle.dump(data, f)

print(f"Đã cập nhật {len(encodings)} khuôn mặt vào: {pkl_file}")
with open("face_encodings.pkl", "rb") as f:
    data = pickle.load(f)
    print("Số khuôn mặt đã lưu:", len(data["encodings"]))
    print("Tên tương ứng:", data["names"])

Đã nạp 200 khuôn mặt đã lưu.
Đang xử lý: 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 0.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 1.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 10.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 100.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 101.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 102.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 103.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 104.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 105.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 106.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 107.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 108.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 109.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 11.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 110.jpg trong 033304000638_TRAN_THI_LINH
Đã lưu encoding cho 111

## test

In [2]:
import face_recognition
import cv2
import pickle
import numpy as np
import os
import csv

# Load dữ liệu encoding
with open("face_encodings.pkl", "rb") as f:
    data = pickle.load(f)

known_encodings = data["encodings"]
known_names = data["names"]

# Mở webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Không mở được webcam.")
    exit()

# Hàm tìm và đọc file .csv từ thư mục có tên folder_name
def read_info_csv(folder_name):
    for root, dirs, files in os.walk("."):
        if os.path.basename(root) == folder_name:
            for file in files:
                if file.endswith(".csv"):
                    with open(os.path.join(root, file), encoding="utf-8") as f:
                        reader = csv.DictReader(f)
                        return next(reader, None)
    return None

recognized_names = set()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc frame từ webcam.")
        break

    frame = cv2.flip(frame, 1)

    # Thu nhỏ ảnh để xử lý nhanh
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]  # Chuyển BGR -> RGB

    # Tìm khuôn mặt
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.5)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        if matches[best_match_index]:
            name = known_names[best_match_index]
            match_score = 1 - face_distances[best_match_index]

            if match_score > 0.7:
                if name not in recognized_names:
                    recognized_names.add(name)
                    info = read_info_csv(name)
                    if info:
                        print(f"== Thông tin của {name} ==")
                        for k, v in info.items():
                            print(f"{k}: {v}")
                    else:
                        print(f"Không tìm thấy file info cho {name}")

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                info_to_scereen  =  f"{name} ({match_score:.2f})" 
                cv2.putText(frame, info_to_scereen, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)


    cv2.imshow("Demo ekyc", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Nhấn ESC để thoát
        break

cap.release()
cv2.destroyAllWindows()


== Thông tin của 019204009589_NGUYEN_THANH_HOAN ==
Họ và tên: NGUYỄN THANH HOÀN
Số thẻ: 019204009589
Ngày sinh: 25/08/2004
Giới tính: Nam
Quốc tịch: Việt Nam
Quê quán: Nga My, Phú Bình, Thái Nguyên
Nơi thường trú: Xóm Thái Hòa, Nga My, Phú Bình, Thái Nguyên
Có giá trị đến: 25/08/2029
